In [1]:
# Experiment 1 - With two output layers and 2 different loss function for leakageness and coordinates. Unable to do hyperparameter tuning since so far i couldnt figure
# out how to do hyper parameter tuning on functional api
# Hence i tried to implement sequential api with 6 outputs and 2 different loss function and do hyper parameter tuning. However i am not able to give 2 loss function 
# in that case. This experiment of giving two different loss function for each output neuron of neural network is being given in custom_loss.ipynb. 

# Experiment 2 - 1 output layer with 1 loss function - mse. and do hyper parameter tuning.
from utils.data_preprocess import load_data, split_xy_save, load_single_leakage_model_data
from utils.module import model_eval, hyper_model, model_comparison, linear_regression, numpy_to_tensor, benchmark_linear_model
import itertools
import pandas as pd 
import yaml
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Model
from keras.layers import Dense, Input
import keras_tuner as kt
from tensorflow import keras
from keras import layers
import seaborn as sns
import matplotlib.pyplot as plt

2.12.0
# GPUs Available:  0


2023-07-01 19:29:00.115055: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [2]:
with open("config_multi.yml", "r") as ymlfile:
    cfg = yaml.full_load(ymlfile)


single_leakage, two_leakage = load_data(total_samples = cfg['experiment']['total_samples'])
two_leakage["leak_1"] = 1
two_leakage["leak_2"] = 1

single_leakage["leak_1"] = 1
single_leakage["leak_2"] = 0

data = pd.concat([single_leakage, two_leakage], axis=0)
data = data.drop(columns=["mfc10_residual"], inplace=False)

data['x2'] = data['x2'] / cfg['geometry']['span']
data['x1'] = data['x1'] / cfg['geometry']['span']

data['y2'] = data['y2'] / cfg['geometry']['width']
data['y1'] = data['y1'] / cfg['geometry']['width']

data['x2'] = data['x2'].replace(np.nan, -5)
data['y2'] = data['y2'].replace(np.nan, -5)
# or try generating a random number

y = data[['x1', 'y1', 'x2', 'y2', 'leak_1', 'leak_2']]
x = data.drop(['x1', 'y1', 'x2', 'y2', 'leak_1', 'leak_2'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1) 

y1_train = y_train[['x1', 'y1', 'x2', 'y2']]
y2_train = y_train[['leak_1', 'leak_2']]
y1_test = y_test[['x1', 'y1', 'x2', 'y2']]
y2_test = y_test[['leak_1', 'leak_2']]
y1_val = y_val[['x1', 'y1', 'x2', 'y2']]
y2_val = y_val[['leak_1', 'leak_2']]

scaler_flows = StandardScaler()
X_train = scaler_flows.fit_transform(X_train)
X_test = scaler_flows.transform(X_test)
X_val = scaler_flows.transform(X_val)


In [3]:
y_train = {
    "y1" : y1_train,
    "y2" : y2_train
}

y_val = {
    "y1" : y1_val,
    "y2" : y2_val
}

y_test = {
    "y1" : y1_test,
    "y2" : y2_test
}

In [4]:
# def loss(y_true, y_pred):
#         a = tf.keras.losses.CategoricalCrossentropy(y_true['leak_1'], y_pred[0]) * y_true['a_1']
#         b = tf.square(y_true['b_0'] - y_pred[1]) * y_true['b_1']
#         b = tf.reduce_sum(b, axis=(1, 2, 3))
#         c = tf.abs(y_true['c_0'] - y_pred[2]) * y_true['c_1']
#         c = tf.reduce_sum(c, axis=1)
#         return a + b + c

In [5]:
losses = {
	"y1": "mse",
	"y2": tf.keras.losses.CategoricalCrossentropy()
    # "y2" : 'mse'
    }

metrics = {
    "y1": 'mae',
    "y2": 'mae'
    }

lossWeights = {"y1": 1.0, "y2": 1.0}

def base_model(inputs):
    # add the sequential layers here
    x= Dense(128, activation='relu', kernel_initializer='he_uniform')(inputs)
    # x= Dense(128, activation='relu', kernel_initializer='he_uniform')(x)
    return x

def final_model(inputs):
    x = base_model(inputs)
    y1 = Dense(units='4', name='y1')(x)
    y2 = Dense(units = '2', name = 'y2')(x)
    model = Model(inputs=inputs, outputs = [y1, y2])
    
    return model

inputs = tf.keras.layers.Input(shape=(10,))
model = final_model(inputs)

model.compile(optimizer=tf.keras.optimizers.Nadam(learning_rate=0.01),
            loss=losses, loss_weights=lossWeights,
            metrics = metrics)

history = model.fit(X_train, y_train, validation_data = (X_val, y_val), verbose = 1, epochs=100, shuffle = True)

Epoch 1/100
12/12 [==============================] - 2s 28ms/step - loss: 11.2342 - y1_loss: 5.3960 - y2_loss: 5.8382 - y1_mae: 1.5981 - y2_mae: 1.3783 - val_loss: 5.1693 - val_y1_loss: 1.7571 - val_y2_loss: 3.4123 - val_y1_mae: 0.9366 - val_y2_mae: 1.5198
Epoch 2/100
12/12 [==============================] - 0s 7ms/step - loss: 3.2775 - y1_loss: 1.2619 - y2_loss: 2.0155 - y1_mae: 0.7523 - y2_mae: 1.8610 - val_loss: 1.6912 - val_y1_loss: 0.8562 - val_y2_loss: 0.8351 - val_y1_mae: 0.5883 - val_y2_mae: 2.4098
Epoch 3/100
12/12 [==============================] - 0s 6ms/step - loss: 1.4593 - y1_loss: 0.6856 - y2_loss: 0.7737 - y1_mae: 0.5337 - y2_mae: 2.7810 - val_loss: 1.4649 - val_y1_loss: 0.7147 - val_y2_loss: 0.7503 - val_y1_mae: 0.5269 - val_y2_mae: 3.1140
Epoch 4/100
12/12 [==============================] - 0s 6ms/step - loss: 1.5083 - y1_loss: 0.5727 - y2_loss: 0.9356 - y1_mae: 0.4773 - y2_mae: 3.2699 - val_loss: 1.4878 - val_y1_loss: 0.5993 - val_y2_loss: 0.8884 - val_y1_mae: 0.4869

In [6]:
model_evaluate, y_pred = model_eval(model, X_test, y_test, X_train, y_train, X_val, y_val)

3/3 [==============================] - 0s 3ms/step - loss: 6.2513 - y1_loss: 0.4046 - y2_loss: 5.8467 - y1_mae: 0.3434 - y2_mae: 4.3397


In [7]:
coords = np.concatenate([y_pred[0], y_test['y1']], axis=1)
presence = np.concatenate([y_pred[1], y_test['y2']], axis=1)

In [8]:
coords

array([[ 0.908,  0.397, -5.104, -5.037,  0.993,  0.449, -5.   , -5.   ],
       [ 0.464,  0.316,  0.759,  0.577,  0.401,  0.306,  0.806,  0.497],
       [ 0.539,  0.509,  0.485,  0.433,  0.51 ,  0.545,  0.634,  0.545],
       [ 0.108, -0.022, -0.016,  0.197,  0.089,  0.067,  0.198,  0.306],
       [ 0.622,  0.037,  1.505,  1.906,  0.946,  0.783,  0.525,  0.927],
       [ 0.371,  0.438, -0.186, -0.192,  0.307,  0.592,  0.852,  0.64 ],
       [ 0.491,  0.743,  0.839,  0.977,  0.556,  0.736,  0.198,  0.927],
       [ 0.133,  0.063, -5.016, -4.923,  0.26 ,  0.21 , -5.   , -5.   ],
       [ 0.486,  0.154, -5.004, -4.96 ,  0.556,  0.306, -5.   , -5.   ],
       [ 0.46 ,  0.664,  0.526,  0.831,  0.556,  0.927,  0.292,  0.449],
       [ 0.081,  0.312,  4.322,  3.832,  0.619,  0.162,  0.307,  0.449],
       [ 0.775,  0.354,  0.477,  0.226,  0.868,  0.736,  0.65 ,  0.21 ],
       [ 0.563,  0.596, -4.957, -4.94 ,  0.743,  0.783, -5.   , -5.   ],
       [ 0.447,  0.387, -4.556, -4.732,  0.572,  0.

In [9]:
presence

array([[ 1.877, -4.255,  1.   ,  0.   ],
       [ 2.77 ,  2.643,  1.   ,  1.   ],
       [ 5.777,  5.476,  1.   ,  1.   ],
       [ 8.576,  7.576,  1.   ,  1.   ],
       [ 7.445,  6.722,  1.   ,  1.   ],
       [ 2.961,  3.344,  1.   ,  1.   ],
       [10.902,  8.316,  1.   ,  1.   ],
       [ 3.517, -5.444,  1.   ,  0.   ],
       [ 1.143, -5.258,  1.   ,  0.   ],
       [ 8.835,  9.074,  1.   ,  1.   ],
       [18.789, 19.68 ,  1.   ,  1.   ],
       [ 3.954,  3.709,  1.   ,  1.   ],
       [ 3.746, -2.052,  1.   ,  0.   ],
       [ 1.688, -3.435,  1.   ,  0.   ],
       [ 5.418, -1.268,  1.   ,  0.   ],
       [ 3.378,  3.772,  1.   ,  1.   ],
       [ 7.918,  7.487,  1.   ,  1.   ],
       [ 3.387, -3.753,  1.   ,  0.   ],
       [ 7.352,  7.408,  1.   ,  1.   ],
       [ 3.988, -4.367,  1.   ,  0.   ],
       [10.615,  9.849,  1.   ,  1.   ],
       [ 4.708,  4.885,  1.   ,  1.   ],
       [ 5.798,  5.236,  1.   ,  1.   ],
       [ 2.274, -3.95 ,  1.   ,  0.   ],
       [ 5.456, 